# Importing the Libraries 

In [ ]:
import pandas as pd
import numpy as np
import os
import re

# ML Commons Scrapping

In [39]:
ml=pd.read_csv(r"C:\Users\Nikhi\Desktop\Files\SIL\mlcommons_word_corpus_final.csv")
ml.shape

C:\Users\Nikhi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(66146, 49)

In [40]:
#printing the statistics of ml commons dataset 
ml.head(100)
ml.shape
ml.columns

Index(['Unnamed: 0', 'english_words', 'Sakha', 'Oriya', 'Dutch', 'Interlingua',
       'Breton', 'Czech', 'Esperanto', 'Catalan', 'Irish', 'Latvian',
       'Vietnamese', 'German', 'Estonian', 'Tatar', 'Lithuanian', 'Frisian',
       'Welsh', 'Slovak', 'Romanian', 'Vallader', 'Hausa', 'Turkish',
       'Russian', 'Maltese', 'Georgian', 'Chuvash', 'Italian', 'Greek',
       'Guarani', 'Portuguese', 'Kyrgyz', 'Slovenian', 'Polish', 'French',
       'Swedish', 'Mongolian', 'Spanish', 'Tamil', 'Chinese', 'Hakha Chin',
       'Persian', 'Indonesian', 'Arabic', 'Ukrainian', 'Sursilvan', 'Basque',
       'Kinyarwada'],
      dtype='object')

In [41]:
#filtering english words
d1=ml[['english_words']]

In [42]:
#Merging with the semantic domains 
os.chdir(r"C:\Users\Nikhi\OneDrive - purdue.edu\Mod 3\Industry Practicum\Files")
#semantic domain file 
semdom = pd.read_csv('semdom_words_eng.csv')
semdom.head()
#Copying the dataset 
df=semdom.copy()
#Dropping the duplicates 
df.drop_duplicates(subset=['word','cids'],inplace=True)
df.shape

(39854, 3)

## Mapping Semantic domains to ML Commons

In [43]:
#merging the file
merged_data = pd.merge(d1,df, left_on='english_words',right_on="word", how='left')

In [44]:
#priting the shape of file
merged_data.shape

(69349, 4)

In [45]:
merged_data['english_words'].isnull().sum()

0

In [46]:
merged_data.drop_duplicates(subset=['english_words'])['cids'].isnull().sum()

63334

In [47]:
merged_data['cids'].isnull().sum()

63334

In [48]:
#taking out the unmerged words 
unmerged_words=merged_data[merged_data['cids'].isnull()]
unmerged_words=unmerged_words[['english_words']]
type(unmerged_words)
unmerged_words=unmerged_words.reset_index()

## Implementation of Word2Vec to map the unmapped words

In [73]:
from gensim.models import KeyedVectors
filename = r"C:\Users\Nikhi\Desktop\Files\SIL\GoogleNews-vectors-negative300.bin.gz"
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [ ]:
#creating an empty list
list_of_words = []
#Empty dictionary
dict_similar={}
#lopping each and every row and create most 50 simlar words to the english words using word2vec
for i in range(len(unmerged_words['word'])):
    try:
        #finding the top 50 similar words
        s=model.similar_by_word(unmerged_words.loc[i,'word'], topn=50)
        #finding the similarity scores also
        dict_similar[unmerged_words.loc[i,'word']]=s
    except:
        list_of_words.append(unmerged_words.loc[i,'word'])
#found similar words in the dictionary
print(dict_similar)
#list of unmapped words after word2vec 
print(list_of_words)

In [ ]:
print(len(dict_similar))

In [ ]:
print(len(list_of_words))
len(list_of_words)

In [ ]:
#domain dictionary
domain_dict={}
#closest word mapping 
mean_dict={}
#lopping in the dictionary
for i,k in dict_similar.items():
    #since k is a list
    for j in k: 
        #Comparing if the words are already there in sematic domain file
        for wd in df['word']:
            #if yes, then find the cid and break the loop
            if wd==j[0]:
                z=0
                if z<=1:
                    domain_dict[i]=list(df[df['word']==j[0]]['cids'])
                    mean_dict[i]=j[0]
                    z=z+1
                else:
                    break
            else:
                continue
print(domain_dict)
print(mean_dict)

In [ ]:
#Converting the final dictionary into dataframe 
word_df1 = pd.DataFrame(columns=['word','cids1'])
#Looping in the dictionary 
for key,value in domain_dict.items():
    #Converting the dictionary into dataframe
    for word in value:
        word_df1 = word_df1.append({'word':key, 'cids1':word}, ignore_index=True)
#printing shape of unmapped words
print(word_df1.shape)
word_df1

In [ ]:
#Finding similar words
mean_df1 = pd.DataFrame(columns=['word','similar_word'])
#looping in to find the similar words
for key,value in mean_dict.items():
    mean_df1 = mean_df1.append({'word':key, 'similar_word':value}, ignore_index=True)
print(mean_df1.shape)
mean_df1

In [ ]:
#finding out the similarity score
for i in range(len(mean_df1)):
    mean_df1.loc[i,'probability']=model.similarity(mean_df1.loc[i,'word'],mean_df1.loc[i,'similar_word'])

In [ ]:
#Basic statistics about the similarity score
mean_df1['probability'].min()
mean_df1['probability'].max()
merged_data.head()

In [50]:
#priting the shape of merged data
print(merged_data.shape)
#merged_data=merged_data.drop(columns=["word"],axis=0)
print(merged_data.shape)
merged_data.columns

(69349, 4)
(69349, 4)


Index(['english_words', 'word', 'category', 'cids'], dtype='object')

## Mapping the similar words to  file

In [ ]:
merged_with_words=pd.merge(merged_data,word_df1,how="left",left_on="english_words",right_on="word")

In [52]:
merged_with_words.head()
merged_data.shape

(69349, 4)

In [53]:
merged_with_words['cids1'].isnull().sum()
merged_with_words.columns
#dropping column 
merged_with_words=merged_with_words.drop(columns=["Unnamed: 0"],axis=1)

In [54]:
merged_with_words['cids1'].isnull().sum()

60673

### Final Mapping with probability

In [59]:
merged_with_words1=pd.merge(merged_with_words,mean_df1,how="left",left_on="english_words",right_on="word")

In [60]:
merged_with_words1=merged_with_words1.drop(columns=['word_x','word_y','Unnamed: 0'])

In [61]:
merged_with_words1

,english_words,category,cids,cids1,word,similar_word,probability
0,1 litre,NaN,NaN,NaN,NaN,NaN,NaN
1,10,NaN,NaN,NaN,NaN,NaN,NaN
2,10 times,NaN,NaN,NaN,NaN,NaN,NaN
3,100,NaN,NaN,NaN,NaN,NaN,NaN
4,100 percent,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
111039,NaN,Name of a thing,9.7.3,NaN,NaN,NaN,NaN
111040,NaN,Name of a thing,9.7.3,NaN,NaN,NaN,NaN
111041,NaN,Name of a thing,9.7.3,NaN,NaN,NaN,NaN
111042,NaN,Name of a thing,9.7.3,NaN,NaN,NaN,NaN


# Final Mapping with all the languages includes

In [62]:
merged_with_words_ml=pd.merge(merged_with_words1,ml,left_on="english_words",right_on="english_words",how="left")

In [63]:
merged_with_words_ml

,english_words,category,cids,cids1,word,similar_word,probability,Unnamed: 0,Sakha,Oriya,...,Tamil,Chinese,Hakha Chin,Persian,Indonesian,Arabic,Ukrainian,Sursilvan,Basque,Kinyarwada
0,1 litre,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,عشرة,NaN,NaN,NaN,NaN
2,10 times,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100 percent,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111039,NaN,Name of a thing,9.7.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111040,NaN,Name of a thing,9.7.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111041,NaN,Name of a thing,9.7.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111042,NaN,Name of a thing,9.7.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Base Dataset including both Panalex and ML Commons

### Importing Panalex final data 

In [70]:
data=pd.read_csv(r"C:\Users\Nikhi\OneDrive - purdue.edu\Mod 3\Industry Practicum\Files\01272022_Final_data_merged.csv",encoding='utf_8_sig')


C:\Users\Nikhi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,6,347,390,489) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [71]:
data.shape

(1192, 784)

In [72]:
data.head()

,Unnamed: 0,index,word,category,cids,cids1,similar_word,pam_language,drl_language,gor_language,...,tkr_language,avi_language,hbn_language,akg_language,fud_language,ebr_language,alp_language,lml_language,hdy_language,mkd_language
0,0,1,I,Pronouns,9.2.3,NaN,NaN,ika/ikayu,nindo/ŋimba/ŋindu,yi7o/yiʔo,...,гъу/йигъын/шу,a/áɣɛ́,ŋa,au,akoe/ke/kee/koe/koutou,hɛ̀,ale,giḡo/xiNg~o,ate/ati/kiʔne,ти
1,1,2,thou,NaN,NaN,NaN,NaN,iya,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ittʼo/itʔo,тој
2,2,2,you,Pronouns,9.2.3,NaN,NaN,iya,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ittʼo/itʔo,тој
3,3,2,you,Prompters of attention,9.6.3.3,NaN,NaN,iya,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ittʼo/itʔo,тој
4,4,5,you,Pronouns,9.2.3,NaN,NaN,ila,-d̪iga,timoŋoliyo,...,мамы/манбы,ɲɔ̃́,nɛdiŋa/ŋindaŋá,duda,kiraua/lotou/lā/lāua/lătou,wò,sie,k|era,issuw/ittʔuw,тие


In [74]:
#Looping through the dataframe 
for i in range(len(data)):
    #checking if the similar word column value is present or not
    if str(data.loc[i,'similar_word'])=="nan":
        #print(1)
        continue
    else:
        #if the similar word column value is present, then we calculate the probability
        try:
            #print(df.loc[i,'similar_word'])
            data.loc[i,'probability']=model.similarity(str(data.loc[i,'word']),str(data.loc[i,'similar_word']))
        except:
            #Checking if the calculation of probability is throwing error
            print(str(data.loc[i,'word']),str(data.loc[i,'similar_word']))

In [76]:
data['probability'].isnull().sum()

1171

In [77]:
#dropping few columns which are not required
data=data.drop(columns=['Unnamed: 0','index'])

In [78]:
#checking the data along with column names 
print(data.columns)
#file above generated in cell 63
merged_with_words_ml.columns

Index(['word', 'category', 'cids', 'cids1', 'similar_word', 'pam_language',
       'drl_language', 'gor_language', 'xbr_language', 'nap_language',
       ...
       'avi_language', 'hbn_language', 'akg_language', 'fud_language',
       'ebr_language', 'alp_language', 'lml_language', 'hdy_language',
       'mkd_language', 'probability'],
      dtype='object', length=783)


Index(['english_words', 'category', 'cids', 'cids1', 'word', 'similar_word',
       'probability', 'Unnamed: 0', 'Sakha', 'Oriya', 'Dutch', 'Interlingua',
       'Breton', 'Czech', 'Esperanto', 'Catalan', 'Irish', 'Latvian',
       'Vietnamese', 'German', 'Estonian', 'Tatar', 'Lithuanian', 'Frisian',
       'Welsh', 'Slovak', 'Romanian', 'Vallader', 'Hausa', 'Turkish',
       'Russian', 'Maltese', 'Georgian', 'Chuvash', 'Italian', 'Greek',
       'Guarani', 'Portuguese', 'Kyrgyz', 'Slovenian', 'Polish', 'French',
       'Swedish', 'Mongolian', 'Spanish', 'Tamil', 'Chinese', 'Hakha Chin',
       'Persian', 'Indonesian', 'Arabic', 'Ukrainian', 'Sursilvan', 'Basque',
       'Kinyarwada'],
      dtype='object')

In [79]:
#renaming the column 
data=data.rename(columns={"word":"english_word"})

In [87]:
#renaming the columns in ML commons dataframe 
ml=merged_with_words_ml.rename(columns={"english_words":"english_word"})
ml['ind']="ML"
data['ind']="Panalex"

In [88]:
#Appending the data into a final dataframe 
final_data=ml.append(data)

In [89]:
#saving the final data 
final_data.to_pickle("Final_data_ml_pan.pkl")
final_data.to_csv("Final_data_ml_pan.csv")

# End of Base data preparation

# Statistics

### These statistics are done to understand the impact of Project deliverables 

In [66]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Nikhi\Desktop\Files\SIL\Final_data_ml_pan.csv")

In [ ]:
df.head()

In [67]:
cids_lang=df[(df['cids1'].isnull())&(df['cids'].isnull())]
cids_lang_cids1=df[df['cids1'].notnull()]
cids_lang_cids=df[df['cids'].notnull()]
cids_lang_cids_check=df[(df['cids'].notnull())&(df['cids1'].notnull())]
print(cids_lang_cids1.shape)
print(cids_lang_cids_check.shape)
print(cids_lang_cids.shape)
print(cids_lang.shape)
print(df.shape)
df.columns.to_list()[7:]

(16553, 834)
(0, 834)
(41015, 834)
(54668, 834)
(112236, 834)


['probability',
 'Unnamed: 0.1',
 'Sakha',
 'Oriya',
 'Dutch',
 'Interlingua',
 'Breton',
 'Czech',
 'Esperanto',
 'Catalan',
 'Irish',
 'Latvian',
 'Vietnamese',
 'German',
 'Estonian',
 'Tatar',
 'Lithuanian',
 'Frisian',
 'Welsh',
 'Slovak',
 'Romanian',
 'Vallader',
 'Hausa',
 'Turkish',
 'Russian',
 'Maltese',
 'Georgian',
 'Chuvash',
 'Italian',
 'Greek',
 'Guarani',
 'Portuguese',
 'Kyrgyz',
 'Slovenian',
 'Polish',
 'French',
 'Swedish',
 'Mongolian',
 'Spanish',
 'Tamil',
 'Chinese',
 'Hakha Chin',
 'Persian',
 'Indonesian',
 'Arabic',
 'Ukrainian',
 'Sursilvan',
 'Basque',
 'Kinyarwada',
 'ind',
 'pam_language',
 'drl_language',
 'gor_language',
 'xbr_language',
 'nap_language',
 'wew_language',
 'uig_language',
 'cmn_language',
 'guk_language',
 'iba_language',
 'cap_language',
 'vro_language',
 'pmf_language',
 'wnk_language',
 'mlv_language',
 'lez_language',
 'erg_language',
 'kem_language',
 'mah_language',
 'tcd_language',
 'zun_language',
 'kod_language',
 'mvo_languag

In [68]:
df_ml=df[df['ind']=="ML"]
cids_lang_ml=df[(df['cids1'].isnull())&(df['cids'].isnull())&(df['ind']=="ML")]
cids_lang_cids1_ml=df[df['cids1'].notnull()&(df['ind']=="ML")]
cids_lang_cids_ml=df[df['cids'].notnull()&(df['ind']=="ML")]
cids_lang_cids_check_ml=df[(df['cids'].notnull())&(df['cids1'].notnull())&(df['ind']=="ML")]
print(cids_lang_cids1_ml.shape)
print(cids_lang_cids_check_ml.shape)
print(cids_lang_cids_ml.shape)
print(cids_lang_ml.shape)
print(df_ml.shape)
df.columns.to_list()[7:]

(16532, 834)
(0, 834)
(39854, 834)
(54658, 834)
(111044, 834)


['probability',
 'Unnamed: 0.1',
 'Sakha',
 'Oriya',
 'Dutch',
 'Interlingua',
 'Breton',
 'Czech',
 'Esperanto',
 'Catalan',
 'Irish',
 'Latvian',
 'Vietnamese',
 'German',
 'Estonian',
 'Tatar',
 'Lithuanian',
 'Frisian',
 'Welsh',
 'Slovak',
 'Romanian',
 'Vallader',
 'Hausa',
 'Turkish',
 'Russian',
 'Maltese',
 'Georgian',
 'Chuvash',
 'Italian',
 'Greek',
 'Guarani',
 'Portuguese',
 'Kyrgyz',
 'Slovenian',
 'Polish',
 'French',
 'Swedish',
 'Mongolian',
 'Spanish',
 'Tamil',
 'Chinese',
 'Hakha Chin',
 'Persian',
 'Indonesian',
 'Arabic',
 'Ukrainian',
 'Sursilvan',
 'Basque',
 'Kinyarwada',
 'ind',
 'pam_language',
 'drl_language',
 'gor_language',
 'xbr_language',
 'nap_language',
 'wew_language',
 'uig_language',
 'cmn_language',
 'guk_language',
 'iba_language',
 'cap_language',
 'vro_language',
 'pmf_language',
 'wnk_language',
 'mlv_language',
 'lez_language',
 'erg_language',
 'kem_language',
 'mah_language',
 'tcd_language',
 'zun_language',
 'kod_language',
 'mvo_languag

In [73]:
count=len(cids_lang_ml['english_word'].value_counts(dropna=True).index)
print(count)
j=0
for i in cids_lang_ml.columns.to_list()[9:]:
    print(i)
    a=len(cids_lang_ml[i].value_counts(dropna=True).index)
    j=j+1
    count=count+a
    print(count,a)
print(j)

54658
Sakha
54658 0
Oriya
54673 15
Dutch
58466 3793
Interlingua
58466 0
Breton
58873 407
Czech
59905 1032
Esperanto
59905 0
Catalan
70681 10776
Irish
71006 325
Latvian
71361 355
Vietnamese
71377 16
German
89112 17735
Estonian
90330 1218
Tatar
90330 0
Lithuanian
90910 580
Frisian
91503 593
Welsh
92814 1311
Slovak
92973 159
Romanian
93224 251
Vallader
93224 0
Hausa
93277 53
Turkish
94626 1349
Russian
98491 3865
Maltese
98491 0
Georgian
98701 210
Chuvash
98701 0
Italian
104614 5913
Greek
104987 373
Guarani
104987 0
Portuguese
107396 2409
Kyrgyz
107396 0
Slovenian
107594 198
Polish
110797 3203
French
126719 15922
Swedish
127060 341
Mongolian
127490 430
Spanish
137820 10330
Tamil
137950 130
Chinese
138293 343
Hakha Chin
138293 0
Persian
143970 5677
Indonesian
144390 420
Arabic
145136 746
Ukrainian
146261 1125
Sursilvan
146261 0
Basque
146261 0
Kinyarwada
146261 0
ind
146262 1
pam_language
146262 0
drl_language
146262 0
gor_language
146262 0
xbr_language
146262 0
nap_language
146262 0
wew_la

146262 0
abs_language
146262 0
mhy_language
146262 0
gaa_language
146262 0
ctz_language
146262 0
tgk_language
146262 0
aut_language
146262 0
ese_language.1
146262 0
art_language.6
146262 0
bph_language.1
146262 0
khm_language
146262 0
tem_language
146262 0
art_language.7
146262 0
vec_language
146262 0
ghn_language
146262 0
heb_language
146262 0
amk_language
146262 0
kyh_language
146262 0
swe_language
146262 0
fuj_language
146262 0
tyv_language
146262 0
hye_language.1
146262 0
ina_language
146262 0
drs_language
146262 0
lmb_language
146262 0
bul_language
146262 0
byn_language
146262 0
jig_language
146262 0
men_language
146262 0
bul_language.1
146262 0
hye_language.2
146262 0
bxr_language
146262 0
ydd_language
146262 0
mri_language
146262 0
pnw_language
146262 0
cym_language
146262 0
mog_language
146262 0
ree_language
146262 0
bxg_language
146262 0
bty_language
146262 0
ote_language
146262 0
sos_language
146262 0
bhw_language
146262 0
ymb_language
146262 0
gin_language
146262 0
myu_langu

146262 0
doo_language
146262 0
aar_language
146262 0
mzh_language
146262 0
htu_language
146262 0
udi_language
146262 0
ssf_language
146262 0
gwa_language
146262 0
yag_language
146262 0
mtp_language
146262 0
nio_language
146262 0
aym_language
146262 0
hat_language
146262 0
tgl_language
146262 0
kkl_language
146262 0
kmn_language
146262 0
ava_language.6
146262 0
kap_language
146262 0
meu_language
146262 0
aba_language
146262 0
btw_language
146262 0
ltz_language
146262 0
pol_language
146262 0
lit_language
146262 0
tjg_language
146262 0
rob_language
146262 0
mbb_language
146262 0
shj_language
146262 0
kzi_language
146262 0
sij_language
146262 0
kum_language
146262 0
ddo_language.1
146262 0
plv_language
146262 0
mph_language
146262 0
yha_language
146262 0
ibd_language
146262 0
lmy_language
146262 0
kap_language.1
146262 0
ava_language.7
146262 0
bkc_language
146262 0
tih_language
146262 0
cji_language
146262 0
cwg_language
146262 0
wbp_language
146262 0
languages_merged.txt_language
146262 

In [75]:
df_pan=df[df['ind']!="ML"]
cids_lang_pan=df[(df['cids1'].isnull())&(df['cids'].isnull())&(df['ind']!="ML")]
cids_lang_cids1_pan=df[df['cids1'].notnull()&(df['ind']!="ML")]
cids_lang_cids_pan=df[df['cids'].notnull()&(df['ind']!="ML")]
cids_lang_cids_check_pan=df[(df['cids'].notnull())&(df['cids1'].notnull())&(df['ind']!="ML")]
print(cids_lang_cids1_pan.shape)
print(cids_lang_cids_check_pan.shape)
print(cids_lang_cids_pan.shape)
print(cids_lang_pan.shape)
print(df_pan.shape)
df.columns.to_list()[7:]

(21, 834)
(0, 834)
(1161, 834)
(10, 834)
(1192, 834)


['probability',
 'Unnamed: 0.1',
 'Sakha',
 'Oriya',
 'Dutch',
 'Interlingua',
 'Breton',
 'Czech',
 'Esperanto',
 'Catalan',
 'Irish',
 'Latvian',
 'Vietnamese',
 'German',
 'Estonian',
 'Tatar',
 'Lithuanian',
 'Frisian',
 'Welsh',
 'Slovak',
 'Romanian',
 'Vallader',
 'Hausa',
 'Turkish',
 'Russian',
 'Maltese',
 'Georgian',
 'Chuvash',
 'Italian',
 'Greek',
 'Guarani',
 'Portuguese',
 'Kyrgyz',
 'Slovenian',
 'Polish',
 'French',
 'Swedish',
 'Mongolian',
 'Spanish',
 'Tamil',
 'Chinese',
 'Hakha Chin',
 'Persian',
 'Indonesian',
 'Arabic',
 'Ukrainian',
 'Sursilvan',
 'Basque',
 'Kinyarwada',
 'ind',
 'pam_language',
 'drl_language',
 'gor_language',
 'xbr_language',
 'nap_language',
 'wew_language',
 'uig_language',
 'cmn_language',
 'guk_language',
 'iba_language',
 'cap_language',
 'vro_language',
 'pmf_language',
 'wnk_language',
 'mlv_language',
 'lez_language',
 'erg_language',
 'kem_language',
 'mah_language',
 'tcd_language',
 'zun_language',
 'kod_language',
 'mvo_languag

In [79]:
count=len(cids_lang_pan['english_word'].value_counts(dropna=True).index)
print(count)
j=0
for i in cids_lang_pan.columns.to_list()[9:]:
    print(i)
    a=len(cids_lang_pan[i].value_counts(dropna=True).index)
    j=j+1
    count=count+a
    print(count,a)
print(j)

10
Sakha
10 0
Oriya
10 0
Dutch
10 0
Interlingua
10 0
Breton
10 0
Czech
10 0
Esperanto
10 0
Catalan
10 0
Irish
10 0
Latvian
10 0
Vietnamese
10 0
German
10 0
Estonian
10 0
Tatar
10 0
Lithuanian
10 0
Frisian
10 0
Welsh
10 0
Slovak
10 0
Romanian
10 0
Vallader
10 0
Hausa
10 0
Turkish
10 0
Russian
10 0
Maltese
10 0
Georgian
10 0
Chuvash
10 0
Italian
10 0
Greek
10 0
Guarani
10 0
Portuguese
10 0
Kyrgyz
10 0
Slovenian
10 0
Polish
10 0
French
10 0
Swedish
10 0
Mongolian
10 0
Spanish
10 0
Tamil
10 0
Chinese
10 0
Hakha Chin
10 0
Persian
10 0
Indonesian
10 0
Arabic
10 0
Ukrainian
10 0
Sursilvan
10 0
Basque
10 0
Kinyarwada
10 0
ind
11 1
pam_language
21 10
drl_language
28 7
gor_language
36 8
xbr_language
44 8
nap_language
54 10
wew_language
62 8
uig_language
72 10
cmn_language
82 10
guk_language
88 6
iba_language
96 8
cap_language
104 8
vro_language
114 10
pmf_language
122 8
wnk_language
130 8
mlv_language
138 8
lez_language
143 5
erg_language
151 8
kem_language
159 8
mah_language
167 8
tcd_language


3685 10
kml_language
3695 10
mtg_language
3703 8
yap_language
3712 9
zyb_language
3721 9
akp_language
3730 9
chd_language
3740 10
jam_language
3748 8
kge_language
3756 8
epo_language
3766 10
liv_language
3776 10
pon_language
3784 8
tay_language
3792 8
yae_language
3800 8
kpw_language
3809 9
iai_language
3817 8
jgo_language
3827 10
cof_language
3836 9
crh_language
3846 10
san_language
3854 8
pru_language
3861 7
bny_language
3869 8
kgp_language
3877 8
ayo_language
3884 7
rou_language
3893 9
kuz_language
3899 6
lmo_language
3908 9
wol_language
3918 10
tdt_language
3926 8
blc_language
3934 8
nds_language
3944 10
kca_language
3952 8
knc_language
3962 10
tbc_language
3971 9
slp_language
3979 8
ita_language.1
3989 10
tuq_language
3997 8
pmt_language
4003 6
plt_language
4013 10
zne_language
4023 10
ljl_language
4031 8
shh_language
4041 10
ajp_language
4051 10
cmi_language
4060 9
sme_language
4070 10
art_language.10
4080 10
nak_language
4087 7
bdy_language.1
4095 8
bzq_language
4104 9
hdn_langu

In [91]:
final_data.head()

,english_word,category,cids,cids1,word,similar_word,probability,Unnamed: 0,Sakha,Oriya,...,tkr_language,avi_language,hbn_language,akg_language,fud_language,ebr_language,alp_language,lml_language,hdy_language,mkd_language
0,1 litre,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10 times,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100 percent,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
final_data.columns

Index(['english_word', 'category', 'cids', 'cids1', 'word', 'similar_word',
       'probability', 'Unnamed: 0', 'Sakha', 'Oriya',
       ...
       'tkr_language', 'avi_language', 'hbn_language', 'akg_language',
       'fud_language', 'ebr_language', 'alp_language', 'lml_language',
       'hdy_language', 'mkd_language'],
      dtype='object', length=833)

In [93]:
final_data.shape

(112236, 833)

In [97]:
cids_lang=final_data[final_data['cids'].notnull()]
cids_lang.columns.to_list()[7:]

['Unnamed: 0',
 'Sakha',
 'Oriya',
 'Dutch',
 'Interlingua',
 'Breton',
 'Czech',
 'Esperanto',
 'Catalan',
 'Irish',
 'Latvian',
 'Vietnamese',
 'German',
 'Estonian',
 'Tatar',
 'Lithuanian',
 'Frisian',
 'Welsh',
 'Slovak',
 'Romanian',
 'Vallader',
 'Hausa',
 'Turkish',
 'Russian',
 'Maltese',
 'Georgian',
 'Chuvash',
 'Italian',
 'Greek',
 'Guarani',
 'Portuguese',
 'Kyrgyz',
 'Slovenian',
 'Polish',
 'French',
 'Swedish',
 'Mongolian',
 'Spanish',
 'Tamil',
 'Chinese',
 'Hakha Chin',
 'Persian',
 'Indonesian',
 'Arabic',
 'Ukrainian',
 'Sursilvan',
 'Basque',
 'Kinyarwada',
 'ind',
 'pam_language',
 'drl_language',
 'gor_language',
 'xbr_language',
 'nap_language',
 'wew_language',
 'uig_language',
 'cmn_language',
 'guk_language',
 'iba_language',
 'cap_language',
 'vro_language',
 'pmf_language',
 'wnk_language',
 'mlv_language',
 'lez_language',
 'erg_language',
 'kem_language',
 'mah_language',
 'tcd_language',
 'zun_language',
 'kod_language',
 'mvo_language',
 'tab_language

In [108]:
count=len(final_data['english_word'].unique())
for i in cids_lang.columns.to_list()[8:]:
    a=len(final_data[i].notnull().unique())
    count=count+a
    print(a)
    print(final_data[i].notnull().sum())
    print(count)
#Value counts 
#

1
0
66332
2
20
66334
2
9876
66336
1
0
66337
2
813
66339
2
4922
66341
1
0
66342
2
21464
66344
2
518
66346
2
864
66348
2
129
66350
2
31070
66352
2
3900
66354
1
0
66355
2
1899
66357
2
1739
66359
2
2192
66361
2
1258
66363
2
1118
66365
1
0
66366
2
254
66368
2
3535
66370
2
11009
66372
1
0
66373
2
475
66375
1
0
66376
2
12646
66378
2
1196
66380
1
0
66381
2
6315
66383
1
0
66384
2
1233
66386
2
11030
66388
2
26771
66390
2
2168
66392
2
1191
66394
2
20539
66396
2
168
66398
2
677
66400
1
0
66401
2
9836
66403
2
1779
66405
2
1993
66407
2
4394
66409
1
0
66410
1
0
66411
1
0
66412
1
112236
66413
2
1178
66415
2
964
66417
2
905
66419
2
905
66421
2
1155
66423
2
898
66425
2
1139
66427
2
1184
66429
2
948
66431
2
905
66433
2
1016
66435
2
1184
66437
2
905
66439
2
898
66441
2
917
66443
2
1076
66445
2
912
66447
2
894
66449
2
901
66451
2
983
66453
2
1084
66455
2
921
66457
2
905
66459
2
1094
66461
2
870
66463
2
929
66465
2
898
66467
2
1076
66469
2
951
66471
2
1154
66473
2
1145
66475
2
905
66477
2
903
66479
2
997
66

1076
67653
2
1080
67655
2
1012
67657
2
1184
67659
2
953
67661
2
985
67663
2
883
67665
2
1166
67667
2
1093
67669
2
1184
67671
2
903
67673
2
950
67675
2
985
67677
2
1076
67679
2
966
67681
2
1044
67683
2
945
67685
2
985
67687
2
1024
67689
2
1065
67691
2
1184
67693
2
1061
67695
2
1104
67697
2
1039
67699
2
919
67701
2
1083
67703
2
939
67705
2
953
67707
2
895
67709
2
905
67711
2
1094
67713
2
1184
67715
2
927
67717
2
1076
67719
2
1092
67721
2
977
67723
2
1090
67725
2
1184
67727
2
1031
67729
2
1138
67731
2
952
67733
2
1075
67735
2
1159
67737
2
963
67739
2
1184
67741
2
936
67743
2
901
67745
2
1184
67747
2
1027
67749
2
908
67751
2
1184
67753
2
911
67755
2
1108
67757
2
1184
67759
2
1143
67761
2
1076
67763
2
1184
67765
2
1005
67767
2
928
67769
2
924
67771
2
1184
67773
2
1184
67775
2
959
67777
2
1184
67779
2
962
67781
2
1162
67783
2
1184
67785
2
907
67787
2
1093
67789
2
889
67791
2
1026
67793
2
914
67795
2
1081
67797
2
927
67799
2
1107
67801
2
896
67803
2
900
67805
2
894
67807
2
905
67809
2
1169
67

### Statistics

In [18]:
data.shape

(1192, 784)

In [19]:
dict1={}
dict2={}
dict3={}
for i in data.columns.to_list():
    dict1[i]=data[i].notnull().count()
    dict2[i]=data[data[i].notnull()]['cids'].count()
    dict3[i]=data[data[i].notnull()]['cids1'].count()

In [21]:
print(dict1)


{'Unnamed: 0': 1192, 'index': 1192, 'word': 1192, 'category': 1192, 'cids': 1192, 'cids1': 1192, 'similar_word': 1192, 'pam_language': 1192, 'drl_language': 1192, 'gor_language': 1192, 'xbr_language': 1192, 'nap_language': 1192, 'wew_language': 1192, 'uig_language': 1192, 'cmn_language': 1192, 'guk_language': 1192, 'iba_language': 1192, 'cap_language': 1192, 'vro_language': 1192, 'pmf_language': 1192, 'wnk_language': 1192, 'mlv_language': 1192, 'lez_language': 1192, 'erg_language': 1192, 'kem_language': 1192, 'mah_language': 1192, 'tcd_language': 1192, 'zun_language': 1192, 'kod_language': 1192, 'mvo_language': 1192, 'tab_language': 1192, 'vot_language': 1192, 'bug_language': 1192, 'nlg_language': 1192, 'dar_language': 1192, 'pue_language': 1192, 'dar_language.1': 1192, 'gwd_language': 1192, 'mee_language': 1192, 'cro_language': 1192, 'mde_language': 1192, 'ilp_language': 1192, 'mxk_language': 1192, 'zlm_language': 1192, 'ndt_language': 1192, 'lez_language.1': 1192, 'pbb_language': 119

In [22]:
print(dict2)


{'Unnamed: 0': 1161, 'index': 1161, 'word': 1161, 'category': 1161, 'cids': 1161, 'cids1': 0, 'similar_word': 0, 'pam_language': 1147, 'drl_language': 937, 'gor_language': 881, 'xbr_language': 881, 'nap_language': 1124, 'wew_language': 874, 'uig_language': 1108, 'cmn_language': 1153, 'guk_language': 922, 'iba_language': 881, 'cap_language': 990, 'vro_language': 1153, 'pmf_language': 881, 'wnk_language': 874, 'mlv_language': 893, 'lez_language': 1050, 'erg_language': 888, 'kem_language': 870, 'mah_language': 877, 'tcd_language': 954, 'zun_language': 1055, 'kod_language': 896, 'mvo_language': 881, 'tab_language': 1065, 'vot_language': 840, 'bug_language': 905, 'nlg_language': 874, 'dar_language': 1050, 'pue_language': 926, 'dar_language.1': 1123, 'gwd_language': 1115, 'mee_language': 881, 'cro_language': 876, 'mde_language': 973, 'ilp_language': 881, 'mxk_language': 942, 'zlm_language': 877, 'ndt_language': 984, 'lez_language.1': 1139, 'pbb_language': 1093, 'tkw_language': 881, 'nij_lang

In [23]:
print(dict3)

{'Unnamed: 0': 21, 'index': 21, 'word': 21, 'category': 0, 'cids': 0, 'cids1': 21, 'similar_word': 21, 'pam_language': 21, 'drl_language': 20, 'gor_language': 16, 'xbr_language': 16, 'nap_language': 21, 'wew_language': 16, 'uig_language': 21, 'cmn_language': 21, 'guk_language': 20, 'iba_language': 16, 'cap_language': 18, 'vro_language': 21, 'pmf_language': 16, 'wnk_language': 16, 'mlv_language': 16, 'lez_language': 21, 'erg_language': 16, 'kem_language': 16, 'mah_language': 16, 'tcd_language': 20, 'zun_language': 21, 'kod_language': 16, 'mvo_language': 16, 'tab_language': 21, 'vot_language': 21, 'bug_language': 16, 'nlg_language': 16, 'dar_language': 21, 'pue_language': 18, 'dar_language.1': 21, 'gwd_language': 21, 'mee_language': 16, 'cro_language': 18, 'mde_language': 17, 'ilp_language': 16, 'mxk_language': 15, 'zlm_language': 16, 'ndt_language': 20, 'lez_language.1': 21, 'pbb_language': 21, 'tkw_language': 16, 'nij_language': 16, 'chv_language': 21, 'kau_language': 20, 'ast_language

# Thank you!